In [ ]:
import numpy as np
import os
from tkinter import Tk, filedialog
from scipy.io import loadmat, savemat
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.palettes import Turbo256
from bokeh.io import output_notebook

# For Jupyter inline display
output_notebook()

In [ ]:
def select_and_load_file():
    """Opens a file dialog and loads a .npz or .mat file."""
    Tk().withdraw()
    file_path = filedialog.askopenfilename(
        filetypes=[("Data files", "*.npz *.mat")], 
        title="Select .npz or .mat file"
    )
    
    if not file_path:
        raise FileNotFoundError("No file selected.")
    
    filename = os.path.splitext(os.path.basename(file_path))[0]

    if file_path.endswith('.npz'):
        data = np.load(file_path, allow_pickle=True)
    elif file_path.endswith('.mat'):
        data = loadmat(file_path, simplify_cells=True)
    else:
        raise ValueError("Unsupported file type.")

    return file_path, filename, data

def list_to_matlab_cell(lst):
    arr = np.empty((1, len(lst)), dtype=object)
    for i, v in enumerate(lst):
        arr[0, i] = v
    return arr

def merge_images(file_path, filename, data):
    """Merges localization sets from the provided data and saves as NPZ and MAT."""
    # Load raw data
    Final_Loc = data['LocalizationsFinal']
    Final_Frame = data['Frame_Information']
    addonarray = data['addonarray']
    cut1array = data['cut1array']
    cut2array = data['cut2array']
    cut3array = data['cut3array']
    Came_from_image = data['Came_from_image']

    max_img_idx = int(np.max(Came_from_image) + 1)
    LocalizationsFinal = [[] for _ in range(max_img_idx)]
    Frame_Information = [[] for _ in range(max_img_idx)]
    Cutsstore = []

    for ksu in range(len(cut1array)):
        print(f"Processing set {ksu + 1}/{len(cut1array)}")

        X1 = Final_Loc[ksu][:, 0]
        X2 = Final_Loc[ksu][:, 1]
        X3 = Final_Loc[ksu][:, 2]
        X4 = Final_Frame[ksu]

        cut1 = cut1array[ksu]
        cut2 = cut2array[ksu]
        cut3 = cut3array[ksu]
        if cut3[1] == 0:
            cut3 = [-1000, 1000]

        Cutsstore_np = np.array(Cutsstore)
        if Cutsstore:
            ssde = np.subtract([cut1[0], cut2[0]], Cutsstore_np)
            ssde2 = np.subtract([cut1[1], cut2[1]], Cutsstore_np)

            if not np.any(np.sum(ssde, axis=1) == 0) and not np.any(np.sum(ssde2, axis=1) == 0):
                IND = np.where((X1 >= cut1[0]) & (X1 <= cut1[1]) &
                               (X2 >= cut2[0]) & (X2 <= cut2[1]) &
                               (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
            elif not np.any(np.sum(ssde, axis=1) == 0):
                IND = np.where((X1 >= cut1[0]) & (X1 < cut1[1]) &
                               (X2 >= cut2[0]) & (X2 < cut2[1]) &
                               (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
            elif not np.any(np.sum(ssde2, axis=1) == 0):
                IND = np.where((X1 > cut1[0]) & (X1 <= cut1[1]) &
                               (X2 > cut2[0]) & (X2 <= cut2[1]) &
                               (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
            else:
                IND = np.where((X1 > cut1[0]) & (X1 < cut1[1]) &
                               (X2 > cut2[0]) & (X2 < cut2[1]) &
                               (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
        else:
            IND = np.where((X1 >= cut1[0]) & (X1 <= cut1[1]) &
                           (X2 >= cut2[0]) & (X2 <= cut2[1]) &
                           (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]

        Cutsstore.append([cut1[1], cut2[1]])
        Cutsstore.append([cut1[0], cut2[0]])

        idx = int(Came_from_image[ksu])
        LocalizationsFinal[idx].extend(np.column_stack((X1[IND], X2[IND], X3[IND])).tolist())
        Frame_Information[idx].extend(X4[IND].tolist())

    # Convert to NumPy arrays
    Final_Loc_Blinking_Corrected = [np.array(loc) for loc in LocalizationsFinal]
    Final_Frame_Blinking_Corrected = [np.array(frames) for frames in Frame_Information]

    # --- Save as NPZ ---
    npz_filename = f'Combined_Final_{filename}.npz'
    np.savez(npz_filename,
             Final_Loc_Blinking_Corrected=np.array(Final_Loc_Blinking_Corrected, dtype=object),
             Final_Frame_Blinking_Corrected=np.array(Final_Frame_Blinking_Corrected, dtype=object))
    print(f"✅ Saved NPZ: {npz_filename}")

    # --- Save as MAT ---
    mat_filename = f'Combined_Final_{filename}.mat'
    savemat(mat_filename, {
        'Final_Loc_Blinking_Corrected': list_to_matlab_cell(Final_Loc_Blinking_Corrected),
        'Final_Frame_Blinking_Corrected': list_to_matlab_cell(Final_Frame_Blinking_Corrected)
    })
    print(f"✅ Saved MAT: {mat_filename}")

    # --- Optional: Visualize first image ---
    if Final_Loc_Blinking_Corrected:
        x = Final_Loc_Blinking_Corrected[0][:, 0]
        y = Final_Loc_Blinking_Corrected[0][:, 1]
        frame_values = Final_Frame_Blinking_Corrected[0]

        source = ColumnDataSource(data=dict(x=x, y=y, frame=frame_values))
        mapper = LinearColorMapper(palette=Turbo256, low=np.min(frame_values), high=np.max(frame_values))

        p = figure(title="Recombined Localizations (Image 1)", match_aspect=True,
                   tools="pan,wheel_zoom,box_zoom,reset,hover,save",
                   tooltips=[("x", "@x"), ("y", "@y"), ("frame", "@frame")])

        p.circle('x', 'y', source=source, size=5,
                 color=linear_cmap('frame', Turbo256, np.min(frame_values), np.max(frame_values)),
                 line_color=None)

        color_bar = ColorBar(color_mapper=mapper, label_standoff=12, location=(0, 0))
        p.add_layout(color_bar, 'right')

        show(p)

In [ ]:
file_path, filename, data = select_and_load_file()
merge_images(file_path, filename, data)